In [1]:
import os

os.chdir('/home/enis/projects/nna/src/nna/exp/megan/run-2/')


In [2]:
# import run
# import nna
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchaudio
torchaudio.set_audio_backend("sox_io")

import numpy as np

from pathlib import Path
from collections import Counter


/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
torch.manual_seed(42)

np.random.seed(42)

torch.backends.cudnn.benchmark = False

In [4]:


loc_per_set = [['45',
  '38',
  '48',
  '39',
  '11',
  '44',
  '46',
  '17',
  '20',
  '50',
  '13',
  '25',
  '21',
  '29',
  '19',
  '16',
  '24',
  '37'],
 ['18', '31', '34', '27', '32', '33', '47', '41', '22', '15'],
 ['30', '12', '14', '36', '40', '49']]


In [6]:
import runconfigs
import wandb
from ignite.metrics import Accuracy, Loss
from ignite.contrib.metrics import ROC_AUC

In [7]:
# from nna.exp import augmentations,
from nna.exp import modelArchs,runutils

In [8]:
# wandb.init(config=runconfigs.default_config, project=runconfigs.PROJECT_NAME)
# config = wandb.config
config = runconfigs.default_config
# wandb.config.update(args) # adds all of the arguments as config variables

params = {
    'batch_size': config['batch_size'],
    'shuffle': True,
    'num_workers': 0
}


In [9]:
device = torch.device(
    f"cuda:{config['device']}" if torch.cuda.is_available() else "cpu")


In [11]:

# labelsbyhumanpath = Path('/scratch/enis/data/nna/labeling/results/')
# sourcePath = Path("/scratch/enis/data/nna/labeling/splits/")


In [12]:
CATEGORY_COUNT = 2


In [13]:
!ls

job_logs  __pycache__  runconfigs.py  start_random_v2-Copy2.ipynb


In [12]:
 # RAW DATA
def load_raw_data(sample_count,CATEGORY_COUNT):
    sample_count = 1200
    X = np.empty((sample_count,480000),dtype=np.float32)
    y_true = np.random.randint(0,CATEGORY_COUNT,(sample_count))
    for i,y in enumerate(y_true):
        X[i,:] = y
    X = np.interp(X, (X.min(), X.max()), (-32768 , 32767))
    n_values = np.max(y_true) + 1
    y_true = np.eye(n_values)[y_true]
    X_train, X_test, y_train, y_test = train_test_split(
                    X, y_true, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
                    X_train, y_train, test_size=0.25,random_state=42)
    return X_train,X_test,X_val,y_train,y_test,y_val

def mock_raw_data(sample_count,CATEGORY_COUNT):
#     sample_count = 30
    X = np.empty((sample_count*CATEGORY_COUNT,480000),dtype=np.float32)
    # y_true = np.random.randint(0,CATEGORY_COUNT,(sample_count))
    y_true=[]
    for i in range(CATEGORY_COUNT):
        y_true.extend([i]*sample_count)
    print(len(y_true))
    for i,y in enumerate(y_true):
        X[i,:] = y
    X = np.interp(X, (X.min(), X.max()), (0 , 3400))
#     X = np.interp(X, (X.min(), X.max()), (-32768 , 32767))
    n_values = np.max(y_true) + 1
    y_true = np.eye(n_values)[y_true]
    X_train, X_test, y_train, y_test = train_test_split(
                    X, y_true, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
                    X_train, y_train, test_size=0.25,random_state=42)
#     print(y_train)
    return X_train,X_test,X_val,y_train,y_test,y_val
    


In [15]:
## Load real data rather than mock 
    # MVP1: delete parts longer than 10 seconds
import run
audio_dataset,_ = run.prepare_dataset()

output_file_path = '/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/files_as_np_filtered_v3_int16.pkl'
audio_dataset.load_audio_files(output_file_path)
audio_dataset.pick_channel_by_clipping()


/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/meganLabeledFiles_wlenV1.txt
/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite
4 files are missing corresponding to excell entries
'-> 5 number of samples are DELETED due to ignore_files and missing_audio_files'
-> 415 samples DELETED because they are not in the excell

-> 0 samples DELETED because they do not have the taxo info coming from excell

-> classes that do not have enough data:
[REMOVED!]
['other-mammal'] 0.0
['other-silence'] 20.0
['unknown-sound'] 2.0
['seabirds'] 1.0
['canids'] 1.0
['other-flare'] 11.0
['other-rain'] 20.0

-> classes that have enough data:
['other-biophony'] 56.0
['other-insect'] 140.0
['other-bird'] 661.0
['songbirds'] 392.0
['duck-goose-swan'] 183.0
['grouse-ptarmigan'] 59.0
['other-anthrophony'] 66.0
['other-aircraft'] 107.0
['loons'] 29.0
['other-car'] 37.0
('-> 102 number of samples are deleted because their taxonomy category does '
 'not have enough data.')
-> classes that do not have

In [16]:
sound_ins=next(iter(audio_dataset.items()))

In [17]:
sound_ins[1].location_id

'11'

In [18]:
# sound_ins[1].taxo_code
# classA = 1.1.7 #'duck-goose-swan']
# classB = 0.2.0 # other-aircraft
sampleTest= []
y=[]
location_id_info = []
expected_len=10
for sound_ins in audio_dataset.values():
    if sound_ins.taxo_code in ['1.1.10','1.1.7']:
        y.append(sound_ins.taxo_code)
        location_id_info.append(sound_ins.location_id)
        if sound_ins.length<10:
            tile_reps = (expected_len/(sound_ins.length)+1)
            repeated_data = np.tile(sound_ins.data,int(tile_reps))
            repeated_data = repeated_data[:expected_len*sound_ins.sr]
            sampleTest.append(repeated_data)
        else:
            sampleTest.append(sound_ins.data[:expected_len*sound_ins.sr])

len(sampleTest),len(y)

# sampleTest=np.array(sampleTest)

(589, 589)

In [19]:
from numpy import argmax
# define input string
# define universe of possible input values
alphabet = ['1.1.10','1.1.7']
# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# integer encode input data
integer_encoded = [char_to_int[char] for char in y]
# print(integer_encoded)
# one hot encode
onehot_encoded = list()
for value in integer_encoded:
	letter = [0 for _ in range(len(alphabet))]
	letter[value] = 1
	onehot_encoded.append(letter)
# print(onehot_encoded)
# invert encoding
inverted = int_to_char[argmax(onehot_encoded[0])]
# print(inverted)
onehot_encoded=np.array(onehot_encoded)

'1.1.10'

In [21]:
X_train,X_test, X_val, y_train, y_test,y_val  = [],[],[],[],[],[]
loc_id_train=[]
loc_id_test=[]
loc_id_valid=[]

for sample,y_val_ins,loc_id in  zip(sampleTest,onehot_encoded,location_id_info):
    if loc_id in loc_per_set[0]:
        X_train.append(sample)
        y_train.append(y_val_ins)
        loc_id_train.append(loc_id)
    elif loc_id in loc_per_set[1]:
        X_test.append(sample)
        y_test.append(y_val_ins)
        loc_id_test.append(loc_id)
    elif loc_id in loc_per_set[2]:
        X_val.append(sample)
        y_val.append(y_val_ins)
        loc_id_valid.append(loc_id)
    else:
        print('error')


In [22]:
set(loc_id_train),set(loc_id_test),set(loc_id_valid)

({'11',
  '16',
  '17',
  '19',
  '20',
  '21',
  '24',
  '25',
  '29',
  '37',
  '38',
  '39',
  '46',
  '48',
  '50'},
 {'15', '18', '22', '27', '31', '33', '47'},
 {'14', '30', '40', '49'})

In [23]:
len(sampleTest)

589

In [24]:
X_train,X_test,X_val=np.array(X_train),np.array(X_test),np.array(X_val)
y_train,y_test,y_val=np.array(y_train),np.array(y_test),np.array(y_val)


In [25]:
# X_train, X_test, y_train, y_test = train_test_split(
#                 sampleTest, onehot_encoded, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(
#                 X_train, y_train, test_size=0.25,random_state=42)

In [26]:
X_train,X_test,X_val=torch.from_numpy(X_train).float(),torch.from_numpy(X_test).float(),torch.from_numpy(X_val).float()
y_train,y_test,y_val=torch.from_numpy(y_train).float(),torch.from_numpy(y_test).float(),torch.from_numpy(y_val).float()


In [27]:
X_train.shape,X_test.shape,X_val.shape

(torch.Size([338, 480000]),
 torch.Size([136, 480000]),
 torch.Size([115, 480000]))

In [28]:
y_train.shape,y_test.shape,y_val.shape

(torch.Size([338, 2]), torch.Size([136, 2]), torch.Size([115, 2]))

In [29]:
# y_val[0:3]

In [30]:
# # X_train,X_test,X_val,y_train,y_test,y_val = load_raw_data('labelsbyhumanpath','sourcePath')
# X_train,X_test,X_val,y_train,y_test,y_val = mock_raw_data(4,CATEGORY_COUNT)
# X_train,X_test,X_val=torch.from_numpy(X_train).float(),torch.from_numpy(X_test).float(),torch.from_numpy(X_val).float()
# y_train,y_test,y_val=torch.from_numpy(y_train).float(),torch.from_numpy(y_test).float(),torch.from_numpy(y_val).float()

# # labelsbyhumanpath = Path('/scratch/enis/data/nna/labeling/results/')
# # with open(labelsbyhumanpath/"np_array_Ymatrix.npy", 'rb') as f:
# #     y_true = np.load(f)

# Counter(np.argmax(y_train,axis=1).tolist()),Counter(np.argmax(y_val,axis=1).tolist()),Counter(np.argmax(y_test,axis=1).tolist())


In [31]:

# X_train2=np.interp(X_train, (X_train.min(), X_train.max()), (-32768 , 32767))
# torch.from_numpy(X_train2).float()


In [32]:
# y_val

In [33]:
# smaple_index=10
# X_train[smaple_index,:],y_train[smaple_index,]


In [34]:
# X_train[0,:].shape

In [35]:
# multiply(torch.ones((1,2)),torch.ones((1,2)))

In [36]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __init__(self, maxMelLen, sampling_rate):
        # sr = 44100 etc
        self.maxMelLen = maxMelLen
        self.sampling_rate = sampling_rate

    def __call__(self, sample):
        x, y = sample
        #https://github.com/PCerles/audio/blob/3803d0b27a4e13efa760227ef6c71d0f3753aa98/test/test_transforms.py#L262
        #librosa defaults
        n_fft = 2048
        hop_length = 512
        power = 2.0
        n_mels = 128
        n_mfcc = 40
        # htk is false in librosa, no setting in torchaudio -?
        # norm is 1 in librosa, no setting in torchaudio -?
        melspect_transform = torchaudio.transforms.MelSpectrogram(sample_rate=self.sampling_rate, window_fn=torch.hann_window,
                                                                  hop_length=hop_length, n_mels=n_mels, n_fft=n_fft)

    
        db_transform = torchaudio.transforms.AmplitudeToDB("power", 80.)
        mel = melspect_transform(x.reshape(-1))
        an_x = db_transform(mel)
        #librosa version
#         mel = librosa.feature.melspectrogram(y=x.reshape(-1),
#                                              sr=self.sampling_rate)
#         an_x = librosa.power_to_db(mel, ref=np.max)
#         an_x = an_x.astype("float32")
#         y = y.astype('float32')
#         print(an_x.shape)
        an_x = an_x[:, :self.maxMelLen]
        # 2-d conv
#         x = an_x.reshape(1, *an_x.shape[:])
        # 1-d conv
        x = an_x.reshape(1, an_x.shape[0]*an_x.shape[1])

        
        return x,y

# #test
# maxMelLen_test = 850
# SAMPLING_RATE_test = 48000
# sample_len_seconds = 10
# # to_tensor works on single sample
# sample_count = 1
# xx_test = torch.ones((sample_count,SAMPLING_RATE_test*sample_len_seconds))
# y_values = torch.ones(sample_count)
# 
# toTensor = ToTensor(maxMelLen_test,SAMPLING_RATE_test)
# x_out,y_out=toTensor((xx_test,y_values))
# x_out.shape,y_out.shape

In [37]:
# X_train,X_test,X_val,y_train,y_test,y_val
X_train[:].shape

torch.Size([338, 480000])

In [38]:
# toTensor = ToTensor(maxMelLen_test,SAMPLING_RATE_test)
# x_out2,y_out=toTensor((X_train[1:2,:],y_train))
# x_out.shape,y_out.shape

In [39]:
# torch.mean(x_out[0]),torch.mean(x_out2[0])
# y_train

In [40]:
# X_train,y_train

In [41]:

# pitch = augmentations.pitch_shift_n_stepsClass(
#     runconfigs.SAMPLING_RATE, config['pitch_shift_n_steps'])
# noise = augmentations.addNoiseClass(config['noise_factor'])
# strech = augmentations.time_stretchClass(runconfigs.SAMPLING_RATE*runconfigs.EXCERPT_LENGTH,
#                                             config['time_stretch_factor'],
#                                             isRandom=True)
# shift = augmentations.shiftClass(config['roll_rate'], isRandom=True)
maxMelLen = 938 # old 850
# toTensor = augmentations.ToTensor(maxMelLen,runconfigs.SAMPLING_RATE)
toTensor = ToTensor(maxMelLen,runconfigs.SAMPLING_RATE)

In [42]:
from ignite.metrics import EpochMetric


def roc_auc_perClass_compute_fn(y_preds, y_targets):
    try:
        from sklearn.metrics import roc_auc_score
    except ImportError:
        raise RuntimeError(
            "This contrib module requires sklearn to be installed.")

    y_true = y_targets.numpy()
    y_pred = y_preds.numpy()
#     print(y_pred,y_true)
#     res = []
#     for y_true_perClass_Index in y_true.shape[1]:
#         res.append(
#             roc_auc_score(y_true[:, y_true_perClass_Index],
#                           y_pred[:, y_true_perClass_Index]))
    res = roc_auc_score(y_true, y_pred, average=None)
    return res


#[docs]
class ROC_AUC_perClass(EpochMetric):
    """Computes Area Under the Receiver Operating Characteristic Curve (ROC AUC)
  accumulating predictions and the ground-truth during an epoch and applying
  `sklearn.metrics.roc_auc_score <http://scikit-learn.org/stable/modules/generated/
  sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score>`_ .

  Args:
      output_transform (callable, optional): a callable that is used to transform the
          :class:`~ignite.engine.engine.Engine`'s ``process_function``'s output into the
          form expected by the metric. This can be useful if, for example, you have a multi-output model and
          you want to compute the metric with respect to one of the outputs.
      check_compute_fn (bool): Optional default False. If True, `roc_curve
          <http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#
          sklearn.metrics.roc_auc_score>`_ is run on the first batch of data to ensure there are
          no issues. User will be warned in case there are any issues computing the function.

  ROC_AUC expects y to be comprised of 0's and 1's. y_pred must either be probability estimates or confidence
  values. To apply an activation to y_pred, use output_transform as shown below:

  .. code-block:: python

      def activated_output_transform(output):
          y_pred, y = output
          y_pred = torch.sigmoid(y_pred)
          return y_pred, y

      roc_auc = ROC_AUC(activated_output_transform)

  """
    def __init__(self,
                 output_transform=lambda x: x,
                 check_compute_fn: bool = False):
#         print(output_transform)
        super(ROC_AUC_perClass,
              self).__init__(roc_auc_perClass_compute_fn,
                             output_transform=output_transform,
                             check_compute_fn=check_compute_fn)


In [43]:

def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    """
  Utility function for computing output of convolutions
  takes a tuple of (h,w) and returns a tuple of (h,w)
  """
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    h = floor(((h_w[0] + (2 * pad) - (dilation *
                                      (kernel_size[0] - 1)) - 1) / stride) + 1)
    w = floor(((h_w[1] + (2 * pad) - (dilation *
                                      (kernel_size[1] - 1)) - 1) / stride) + 1)
    return h, w

# mel.shape,an_x.shape,X_train.shape
class testModel(nn.Module):
    '''A simple model for testing by overfitting.
    '''
    def __init__(self, out_channels, h_w, kernel_size, FLAT=False,output_shape=(10,)):
        # h_w: height will be always one since we use 1d convolution 
        super(testModel, self).__init__()
        self.out_channels = out_channels
        #### CONV
        self.conv1 = nn.Conv1d(in_channels=1, # depth of image == depth of filters
                               out_channels=self.out_channels, # number of filters 
                               kernel_size=kernel_size, # size of the filters/kernels
                               padding=1)

        self.conv1_shape = conv_output_shape(h_w, kernel_size=kernel_size, stride=1, pad=1, dilation=1)
        # conv is 1d
        self.conv1_shape = (1,self.conv1_shape[1])
        
        self.fc1 = nn.Linear(self.out_channels * self.conv1_shape[0] *self.conv1_shape[1], 64)  # 100

        self.fc2 = nn.Linear(64,output_shape[0])
        
    def forward(self, x):
#         x = x.reshape(1,)
#         print(x.shape) #  50,1,108800 (850*128)
        x = F.relu(self.conv1(x))
#         x = self.pool(x)
        # x = self.drop(x)
#         print(x.shape)# 58, 2, 108801
#         print(self.conv1_shape)
#         print(x.shape)
        x = x.view(-1, self.out_channels * self.conv1_shape[0] *self.conv1_shape[1])
        # batch_norm is missing
        x = F.relu((self.fc1(x)))
        x = (self.fc2(x))

#         x = self.drop(x)

#         x = self.fc4(x)
#         x = torch.sigmoid(x)
#                 x = F.log_softmax(x,dim=1)
        return x

# test
# input_shape=(1,(938*128))
# output_shape=(10,)
# testModel_ins=adam(out_channels=2,h_w=input_shape,kernel_size=2,output_shape=output_shape)
# # a.conv1.weight
# a_out=testModel_ins(torch.ones((3,1,input_shape[1])))

# a_out_correct=torch.zeros(a_out.shape)
# a_out_correct[0][:]=1
# a_out_correct
# a_out.detach().numpy()

# torch.exp(a_out),a_out

In [44]:

# for y_true_perClass_Index in a_out_correct.shape[1]:
#     print(y_true_perClass_Index)

In [45]:
# from ignite.contrib.metrics import ROC_AUC
# from nna.exp.metrics import ROC_AUC_perClass
def activated_output_transform(output):
    y_pred, y = output
#     y_pred = torch.exp(y_pred)
    return y_pred, y

In [46]:
# asd=ROC_AUC_perClass(activated_output_transform)
# asd.update((a_out,a_out_correct))
# asd.compute()

In [47]:

transformCompose = transforms.Compose([
#     pitch,
#     strech,
#     shift,
#     noise,
    toTensor,
])


sound_datasets = {
    phase: runutils.audioDataset(XY[0], XY[1], transform=transformCompose)
    for phase, XY in
    zip(['train', 'val', 'test'],
        [[X_train, y_train], [X_val, y_val], [X_test, y_test]])
}

dataloaders = {
    x: torch.utils.data.DataLoader(sound_datasets[x], **params)
    for x in ['train', 'val', 'test']
}

# this will change
h_w = [128, 938]
kernel_size = (4, 4)
# if config['CNNLayer_count'] == 1:
#     model = modelArchs.NetCNN1(config['CNN_filters_1'], h_w,
#                                 kernel_size).float().to(device)

# if config['CNNLayer_count'] == 2:
#     model = modelArchs.NetCNN2(config['CNN_filters_1'], config.CNN_filters_2,
#                                 h_w, kernel_size,
#                                 kernel_size).float().to(device)

#simpler model

output_shape=(CATEGORY_COUNT,)
model = testModel(out_channels=2,h_w=(1,h_w[0]*h_w[1]),kernel_size=kernel_size[0]*kernel_size[0],output_shape=output_shape)
model.float().to(device)

# device is defined before

model.float().to(device)  # Move model before creating optimizer
optimizer = torch.optim.AdamW(model.parameters(),
#                                 weight_decay=config['weight_decay'],
                             )



criterion = nn.BCEWithLogitsLoss()
# statHistory={'valLoss':[],'trainLoss':[],'trainAUC':[],'valAUC':[]}

metrics = {
    'loss': Loss(criterion),  # 'accuracy': Accuracy(),
#     'ROC_AUC': ROC_AUC(runutils.activated_output_transform),
    'ROC_AUC': ROC_AUC_perClass(activated_output_transform),

}


In [48]:
model

testModel(
  (conv1): Conv1d(1, 2, kernel_size=(16,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=240102, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

In [49]:
model.conv1.weight

Parameter containing:
tensor([[[ 0.1911,  0.2075, -0.0586,  0.2297, -0.0548,  0.0504, -0.1217,
           0.1468,  0.2204, -0.1834,  0.2173,  0.0468,  0.1847,  0.0339,
           0.1205, -0.0353]],

        [[ 0.1927,  0.0370, -0.1167,  0.0637, -0.1152, -0.0293, -0.1015,
           0.1658, -0.1973, -0.1153, -0.0706, -0.1503,  0.0236, -0.2469,
           0.2258, -0.2124]]], device='cuda:1', requires_grad=True)

In [50]:
model.conv1

Conv1d(1, 2, kernel_size=(16,), stride=(1,), padding=(1,))

In [51]:
model.fc1.weight

Parameter containing:
tensor([[-6.6266e-04,  1.2611e-03,  3.1806e-04,  ..., -1.1344e-03,
          1.7489e-04, -2.2777e-05],
        [-1.3098e-03, -1.8677e-03, -6.3242e-04,  ..., -1.6664e-03,
          6.8555e-04,  1.7273e-03],
        [-1.5009e-03,  8.8956e-04,  6.2944e-04,  ...,  1.2712e-03,
          1.2644e-03,  1.4021e-03],
        ...,
        [ 1.1005e-03, -1.7050e-03,  5.0363e-04,  ...,  1.6579e-03,
          5.5793e-05,  1.2571e-04],
        [-1.6759e-03,  1.6106e-03,  9.2794e-04,  ..., -1.7791e-03,
          1.6985e-03, -8.0668e-04],
        [-2.7139e-04, -1.2249e-03,  3.8873e-04,  ...,  1.9329e-03,
          1.8754e-03, -1.3147e-03]], device='cuda:1', requires_grad=True)

In [53]:
# no init 16, 64
print('ready ?')
runutils.run(model, dataloaders, optimizer, criterion, metrics, device,config, runconfigs.PROJECT_NAME)


ready ?


2021-01-26 04:18:04,838 Trainer INFO: Engine run starting with max_epochs=2000.
/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/aten/src/ATen/native/SpectralOps.cpp:590.)
  normalized, onesided, retu

train loss 1809.624494383321
val loss 4160.851503057065
validation roc auc [0.37904229 0.38152985] 1
train roc auc [0.46339587 0.46351423] 1


2021-01-26 04:18:24,097 Train Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-26 04:18:28,436 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:04
2021-01-26 04:18:28,438 Train Evaluator INFO: Engine run complete. Time taken: 00:00:04
2021-01-26 04:18:28,439 Val Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-26 04:18:29,755 Val Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:01
2021-01-26 04:18:31,051 Val Evaluator INFO: Engine run complete. Time taken: 00:00:03
wandb: WARNING Step must only increase in log calls.  Step 2 < 12; dropping {'lr/group_0': 0.001}.
2021-01-26 04:18:31,064 Trainer INFO: Epoch[2] Complete. Time taken: 00:00:11


train loss 1375.8434061806583
val loss 1737.8745446246603
validation roc auc [0.38059701 0.63681592] 2
train roc auc [0.46369178 0.53719595] 2


...

train loss 0.0016502829133560495
val loss 1.814799692319787
validation roc auc [0.72947761 0.62406716] 336
train roc auc [1. 1.] 336


2021-01-26 05:26:43,989 Train Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-26 05:26:49,485 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:05
2021-01-26 05:26:49,488 Train Evaluator INFO: Engine run complete. Time taken: 00:00:05
2021-01-26 05:26:49,489 Val Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-26 05:26:50,978 Val Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:01
2021-01-26 05:26:50,980 Val Evaluator INFO: Engine run complete. Time taken: 00:00:01
wandb: WARNING Step must only increase in log calls.  Step 337 < 2022; dropping {'lr/group_0': 0.001}.
2021-01-26 05:26:50,987 Trainer INFO: Epoch[337] Complete. Time taken: 00:00:12


train loss 0.001634989183164648
val loss 1.7169181668240092
validation roc auc [0.7300995  0.62779851] 337
train roc auc [1. 1.] 337


2021-01-26 05:26:55,876 Train Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-26 05:26:58,664 Train Evaluator ERROR: Engine run is terminating due to exception: .
2021-01-26 05:26:58,666 Trainer ERROR: Engine run is terminating due to exception: .


KeyboardInterrupt: 

In [ ]:
model.fc1.weight

In [ ]:

example_input = (torch.ones((1,1,h_w[0]*h_w[1]))*-32767)
example_input = example_input.float().to(device)
out=model(example_input)
out

activated_output_transform((out,out))

In [ ]:
torch.Tensor([10,10,10])

In [ ]:
sum(model.fc1.weight)

In [ ]:
m = nn.Sigmoid()

loss = nn.BCEWithLogitsLoss()
input = torch.tensor([100.0,100,100], requires_grad=True)
# target = torch.empty(3).random_(2)
target = torch.ones(3)
output = loss((input), target)
input,target,output

In [ ]:
input

In [ ]:
target

In [ ]:
output

In [ ]:
m(input)